In [5]:
pip install facenet-pytorch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from facenet_pytorch import InceptionResnetV1

c:\Users\julie\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Ce modèle implémente ce qui est décrit dans "FaceNet: A Unified Embedding for Face Recognition and Clustering". Cependant il est pré-entraîné donc il faudrait le retrain grâce à des triplets fait sur base des chimpanzés qu'on doit reconnaître. 

In [ ]:
model = InceptionResnetV1(pretrained='casia-webface', classify=False)

100%|██████████| 111M/111M [00:11<00:00, 10.0MB/s] 


Modèle chargé avec succès !


Voilà comment réentrainer le model grâce au dataset de chimpanzé. Il faut juste modifier la création des triplets pour utiliser une des techniques mentionnées dans l'article (j'opterais plus pour la méthode dynamique)

In [ ]:
import torch
from torch.utils.data import DataLoader
from facenet_pytorch import InceptionResnetV1, MTCNN
from torchvision import datasets, transforms
from facenet_pytorch import models
from facenet_pytorch import training

# Charger le modèle pré-entraîné InceptionResnetV1 sans classification
model = InceptionResnetV1(pretrained='casia-webface', classify=False).eval()

# Préparer les transformations pour les images (normalisation des pixels, redimensionnement)
transform = transforms.Compose([
    transforms.Resize((160, 160)),  # Redimensionner les images à 160x160
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalisation
])

# Charger le dataset (remplacer le chemin par le vôtre)
dataset = datasets.ImageFolder(root='/path/to/your/dataset', transform=transform)

# Créer un DataLoader pour itérer sur le dataset pendant l'entraînement
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Fonction de création de triplets
# Cette fonction crée des triplets à partir des images du dataset
def create_triplets(dataset, batch_size=32):
    # Exemple de génération de triplets. Vous pouvez personnaliser cette fonction
    # pour la rendre plus efficace et adaptée à vos besoins
    triplets = []
    for i in range(0, len(dataset), batch_size):
        anchor_image = dataset[i]
        positive_image = dataset[i + 1]  # Trouver un positif dans le même sous-dossier
        negative_image = dataset[(i + 2) % len(dataset)]  # Trouver un négatif dans un sous-dossier différent
        triplets.append((anchor_image, positive_image, negative_image))
    return triplets

# Créer une boucle d'entraînement
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 10

for epoch in range(num_epochs):
    model.train()  # Mode entraînement
    for batch in dataloader:
        images, _ = batch  # Obtenir les images
        triplets = create_triplets(images)  # Créer des triplets à partir du lot

        # Séparer les triplets en ancres, positifs et négatifs
        anchor, positive, negative = triplets

        # Calculer les embeddings
        anchor_embedding = model(anchor)
        positive_embedding = model(positive)
        negative_embedding = model(negative)

        # Calculer la triplet loss
        loss = torch.nn.functional.triplet_margin_loss(anchor_embedding, positive_embedding, negative_embedding, margin=1.0)

        # Mise à jour des poids
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

# Sauvegarder le modèle fine-tuné
torch.save(model.state_dict(), 'retrained_model.pth')
